In [1]:
# # Environment variable
# %env XLA_PYTHON_CLIENT_ALLOCATOR=platform

In [2]:
# # --- Standard Python Libraries ---
# import os
# import multiprocessing
# import time
# # --- Core Scientific & Data Libraries ---
# import numpy as np
# import jax.numpy as jnp
# import h5py
# from scipy.interpolate import griddata
# from scipy.stats import qmc

# # --- Visualization & Image Processing ---
# import matplotlib.pyplot as plt
# import matplotlib.colors as mcolors
# from skimage import measure
# from tqdm import tqdm

# # --- Your Local Project Modules ---
# # Note: It's often better to import specific functions instead of using '*'
# from analyze import *
# from pinn_toolkit.util import *

In [3]:
# def analyze_solution_in_batch(h5file):
#     '''
#     takes in a h5file name (assume the file exist under the current directory)
#     and analyze the solution properties such as curvature, arc length
#     '''
#     start = time.time()
#     result = batch_analyze(h5file)
#     print(f"time elapsed is {(time.time() - start):.2f} s")
#     return result

In [4]:
# filename = os.path.join("data","2rqmc_5k_128*128")
# analysis_results = analyze_solution_in_batch(filename)

In [5]:
# demo(filename, 12634)

In [6]:
# # --- create plot directory ---
# out_dir = os.path.join("plots", "solution_properties")
# os.makedirs(out_dir, exist_ok=True)
# # --- create plot name ---
# base_name = os.path.basename(filename)
# plot_name, _ = os.path.splitext(base_name)
# # --- make the plot --- 
# make_pde_property_heatmaps(analysis_results, out_dir, plot_name)

In [7]:
# import numpy as np
# import matplotlib.pyplot as plt
# import matplotlib.colors as mcolors
# from scipy.interpolate import griddata
# from scipy.stats import qmc

# def sample_from_heatmap(x, y, z, n_samples: int, n_contour_lines: int = 10,
#                         grid_resolution: int = 200, nlevels: int = 100):
#     """
#     Performs proportional stratified sampling based on the area of z-value strata.

#     This function defines strata as the regions between contour lines. It calculates
#     the area of each stratum and allocates samples proportionally, so larger areas
#     receive more samples. It uses a QMC sequence to ensure samples are well-distributed
#     within each stratum.

#     Args:
#         x (array-like): The x-coordinates of the original data points.
#         y (array-like): The y-coordinates of the original data points.
#         z (array-like): The z-values for the heatmap.
#         n_samples (int): The total number of samples to draw.
#         n_contour_lines (int): The number of contour lines to define the strata.
#         grid_resolution (int): The resolution of the interpolated heatmap grid.
#         nlevels (int): The number of color levels for the heatmap visualization.

#     Returns:
#         np.ndarray or None: An array of the sampled (x, y) coordinates.
#     """
#     title_str = "Heatmap with Proportional Stratified QMC Samples"
#     print(f"Generating heatmap and performing proportional stratified sampling...")
    
#     x, y, z = np.asarray(x), np.asarray(y), np.asarray(z)

#     # --- Data Validation ---
#     valid_mask = ~np.isnan(x) & ~np.isnan(y) & ~np.isnan(z)
#     x, y, z = x[valid_mask], y[valid_mask], z[valid_mask]
#     if len(z) < 3:
#         print("Error: Not enough valid data points for interpolation.")
#         return None

#     # --- Grid Interpolation ---
#     # This is done once and reused for everything
#     grid_res_complex = complex(0, grid_resolution)
#     grid_x, grid_y = np.mgrid[x.min():x.max():grid_res_complex, y.min():y.max():grid_res_complex]
#     grid_z = griddata((x, y), z, (grid_x, grid_y), method='linear')
    
#     # Filter out NaN values in the grid that are outside the convex hull of the data
#     valid_grid_mask = ~np.isnan(grid_z)
    
#     # --- Define Strata and Calculate Proportions ---
#     print("  - Defining strata and calculating their relative areas...")
#     vmin, vmax = np.nanmin(grid_z), np.nanmax(grid_z)
    
#     # Use the contour levels to define the boundaries of our strata
#     contour_levels = np.linspace(vmin, vmax, n_contour_lines + 1)
#     strata_bounds = np.unique(np.concatenate(([vmin], contour_levels, [vmax])))
#     n_strata = len(strata_bounds) - 1

#     strata_info = []
#     total_valid_cells = 0
    
#     # First pass: find all coordinates and the size of each stratum
#     for i in range(n_strata):
#         lower_bound, upper_bound = strata_bounds[i], strata_bounds[i+1]
        
#         # Find grid cells within this stratum's z-value range
#         mask = (grid_z >= lower_bound) & (grid_z < upper_bound) & valid_grid_mask
        
#         # Get the (x, y) coordinates for these valid grid points
#         coords = np.vstack((grid_x[mask], grid_y[mask])).T
        
#         stratum_size = len(coords)
#         if stratum_size > 0:
#             strata_info.append({'coords': coords, 'size': stratum_size})
#             total_valid_cells += stratum_size

#     # --- Proportional Sample Allocation ---
#     print("  - Allocating samples proportionally to stratum area...")
#     ideal_samples = []
#     for stratum in strata_info:
#         proportion = stratum['size'] / total_valid_cells
#         stratum['proportion'] = proportion
#         ideal_samples.append(proportion * n_samples)

#     # Deterministically allocate integer number of samples
#     samples_to_draw = [int(s) for s in ideal_samples]
#     remainder = n_samples - sum(samples_to_draw)
#     if remainder > 0:
#         fractional_parts = [s - int(s) for s in ideal_samples]
#         remainder_indices = np.argsort(fractional_parts)[-remainder:]
#         for idx in remainder_indices:
#             samples_to_draw[idx] += 1

#     # --- Perform Sampling ---
#     print("  - Drawing samples from each stratum...")
#     all_samples = []
#     for i, stratum in enumerate(strata_info):
#         n_to_draw = min(samples_to_draw[i], stratum['size'])
        
#         if n_to_draw > 0:
#             # Use a 1D QMC sampler to pick well-distributed INDICES
#             sampler = qmc.Halton(d=1, scramble=True)
#             qmc_indices_float = sampler.random(n=n_to_draw).flatten()
#             qmc_indices = (qmc_indices_float * stratum['size']).astype(int)
#             unique_indices = np.unique(qmc_indices)
            
#             all_samples.append(stratum['coords'][unique_indices])

#     samples_array = np.concatenate(all_samples, axis=0) if all_samples else np.array([[]])

#     # --- VISUALIZATION ---
#     fig, ax = plt.subplots(figsize=(10, 8))
#     ax.grid(False)
    
#     # Plot the heatmap and contours
#     cmap = plt.get_cmap('viridis')
#     levels = np.linspace(vmin, vmax, nlevels)
#     norm = mcolors.BoundaryNorm(boundaries=levels, ncolors=cmap.N)
#     im = ax.imshow(grid_z.T, extent=(x.min(), x.max(), y.min(), y.max()),
#                    origin='lower', cmap=cmap, norm=norm, aspect='auto')
#     ax.contour(grid_x, grid_y, grid_z, levels=contour_levels,
#                colors='white', linewidths=0.7, alpha=0.6)

#     # Overlay the proportionally stratified samples
#     ax.scatter(samples_array[:, 0], samples_array[:, 1], 
#                c='red', s=30, edgecolor='black', linewidth=0.8,
#                label=f'Sampled Points ({len(samples_array)})')
#     ax.legend()

#     # --- Formatting and Display ---
#     ax.set_title(title_str, fontsize=20, pad=15)
#     ax.set_xlabel("X-axis", fontsize=16, labelpad=10)
#     ax.set_ylabel("Y-axis", fontsize=16, labelpad=10)
#     cbar = fig.colorbar(im, ax=ax, pad=0.02)
#     cbar.set_label("Z-value", fontsize=16, labelpad=10)
#     plt.show()

#     return samples_array

In [8]:
# analysis_results.keys()

In [9]:
# x = analysis_results['L_pde']
# y = analysis_results['M_pde']
# z = analysis_results['arc_length']

In [10]:
# n_contour = 10
# n_samples = 25
# samples_array = sample_from_heatmap(x,y,z,nlevels=100, n_contour_lines = n_contour, n_samples = n_samples)
# save_pytree(jnp.asarray(samples_array), f"params_{n_contour+1}_{n_samples}")

In [11]:
# def plot1(x, y, z, n=10):
#     import numpy as np
#     import matplotlib.pyplot as plt
#     import matplotlib.tri as tri
#     from matplotlib.colors import ListedColormap

#     print(f"--- Plotting {n} quantile regions with distinct colors ---")

#     # --- Data Validation ---
#     x, y, z = np.asarray(x), np.asarray(y), np.asarray(z)
#     valid_mask = ~np.isnan(x) & ~np.isnan(y) & ~np.isnan(z)
#     x, y, z = x[valid_mask], y[valid_mask], z[valid_mask]

#     if len(z) < 3:
#         print("Error: Not enough valid data points to create a triangulation.")
#         return

#     # --- Compute quantile bins ---
#     quantiles = np.linspace(0, 1, n + 1)
#     quantile_values = np.quantile(z, quantiles)

#     z_bin = np.zeros_like(z, dtype=int)
#     for i in range(1, len(quantile_values)):
#         mask_i = (quantile_values[i - 1] < z) & (z <= quantile_values[i])
#         z_bin[mask_i] = i

#     # --- Triangulation ---
#     triang = tri.Triangulation(x, y)

#     # --- Distinct colormap ---
#     base_cmap = plt.cm.get_cmap('tab20')  # 20 categorical colors
#     colors = [base_cmap(i % 20) for i in range(n)]

#     # Reorder colors to maximize contrast between neighbors
#     if n > 2:
#         colors = [colors[i % len(colors)] for i in np.argsort(np.arange(n) * 7 % n)]

#     cmap = ListedColormap(colors)

#     # --- Plot ---
#     fig, ax = plt.subplots(figsize=(8, 6))
#     ax.grid(False)

#     tpc = ax.tripcolor(
#         triang,
#         z_bin,
#         cmap=cmap,
#         shading="flat",
#         vmin=1,
#         vmax=n
#     )

#     # --- Colorbar ---
#     cbar = plt.colorbar(tpc, ax=ax)
#     tick_locs = np.arange(1, n + 1)
#     cbar.set_ticks(tick_locs)
#     cbar.set_ticklabels([f"Q{i}" for i in tick_locs])
#     cbar.set_label("Quantile Bin", fontsize=12)

#     # --- Labels ---
#     ax.set_title(f"Quantile Visualization ({n} bins, distinct colors)", fontsize=16, pad=10)
#     ax.set_xlabel("X-axis", fontsize=14)
#     ax.set_ylabel("Y-axis", fontsize=14)
#     ax.set_xlim(x.min(), x.max())
#     ax.set_ylim(y.min(), y.max())

#     plt.tight_layout()
#     plt.show()


In [12]:
# i = 8 # select from 1 to 9
# # mask = (quantile_values[i-1] < z) & (z <= quantile_values[i])
# # x_sub = x[mask]
# # y_sub = x[mask]
# plot1(x,y,z,n=4)